# 五. `softmax`回归

## 1. 概述

`Softmax`回归（`Softmax Regression`），也称为多项（`Multinomial`）或多类（`Multi-Class`）的`Logistic`回归，是`Logistic`回归在多分类问题上的推广。

对于多类问题，类别标签$y \in {1, 2,..., C}$ 可以有$C$个取值．给定一个样本$x$，`Softmax` 回归预测的属于类别$c$的条件概率为
$$
\begin{aligned}
p(y=c|\mathbf{x})&=\mathrm{softmax}(\mathbf{w^T_c x})\\
&=\frac{\exp(\mathbf{w^T_c x})}{\sum_{i=1}^C \exp(\mathbf{w^T_i x})}
\end{aligned}
$$

其中$\mathbf{w_i}$是第i类的权重向量。

In [ ]:
%config InlineBackend.figure_formats = ['svg']
%matplotlib inline
from torch.utils.data import TensorDataset, DataLoader
import torch
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
def softmax(X, W):
    """
    X: torch.FloatTensor, N*a, N样本数量, a为特征的维度
    W: torch.FloatTensor, a*C, C为类别数量
    """
    C = torch.exp(X@W)  # hat_y, N*C
    return C / torch.sum(C, axis=1).reshape(X.shape[0], -1)  # 各样本对应类别的标准化概率分布

In [ ]:
X = torch.randn(10, 5)
W = torch.randn(5, 3)
softmax(X, W)

## 2. Softmax回归的决策函数

`Softmax`回归的决策函数可以表示为

$$
\begin{aligned}
\hat{y}&=\text{arg}\max_{i=1}^{C}p(y=c|\mathbf{x})\\
&=\text{arg}\max_{i=1}^{C}\mathbf{w_i^T x}
\end{aligned}
$$

In [ ]:
def hat_y(X, W):
    S = softmax(X, W)  # 各样本在各类别上的概率
    max_indices = torch.max(S, dim=1)[1]
    pred_y = torch.zeros_like(S)
    pred_y[torch.arange(S.shape[0]), max_indices] = 1  # one-hot
    return max_indices, pred_y

In [ ]:
pred_y = hat_y(X, W)

In [ ]:
pred_y[0]

In [ ]:
pred_y[1]

- 与`Logistic`回归的关系。当类别数$C=2$时，softmax回归的决策函数为
$$
\begin{aligned}
\hat{y}&=\text{arg}\max_{i\in\{1,2\}}p(y=c|\mathbf{x})\\
&=\text{arg}\max_{i\in\{1,2\}}\mathbf{w_i^T x}\\
&=I(\mathbf{(w_2-w_1)^T x}>0))
\end{aligned}
$$
其中$I(\cdot)$是指示函数。

## 3. 准则

给定`N`个训练样本，`Softmax`回归使用**交叉熵损失函数**学习最有的参数矩阵$W$。为了方便起见，使用$C$维的`one-hot`向量表示类别标签，对于类别$i$，其向量表示为
$$
y = [I(i=1), I(i=2), ..., I(i=C)]
$$

采用交叉熵损失函数，`Softmax`回归模型的风险函数是
$$
\begin{aligned}
R(\mathbf{W})&=-\frac{1}{N}\sum_{n=1}^N\sum_{i=1}^{C}y_i^{(n)}\log \hat{y}_i^{(n)}\\
&=-\frac{1}{N}\sum_{n=1}^N(\mathbf{y^{(n)}})^T\log \hat{y}_c^{(n)}
\end{aligned}
$$

其中，$\hat{y}_c^{(n)}=\text{softmax}(\mathbf{W^T x^{(n)}})$ 为样本 $x^{(n)}$ 在每个类别的后验概率。

- 交叉熵

交叉熵损失函数是一种常用于机器学习分类问题的损失函数。我们可以从它解决的问题和它如何工作的基本概念理解它。

首先，当我们在做分类任务时，我们的目标是让模型能够准确地预测输入数据的类别。例如，在一个简单的二分类问题中，我们可能想要预测一个电子邮件是“垃圾邮件”还是“非垃圾邮件”。

在这种情况下，模型会为每个类别输出一个概率，这些概率的总和为1。交叉熵损失函数帮助我们衡量模型预测的概率分布与真实情况的概率分布之间的差异。理想情况下，如果模型的预测完全准确，它预测的概率分布将与实际情况完全一致。

交叉熵损失函数的计算方式是这样的：对于给定的真实类别，我们只关注该类别对应的模型输出概率。如果真实类别是“垃圾邮件”（我们可以用1表示），我们就查看模型预测“垃圾邮件”这一类别的概率；如果真实类别是“非垃圾邮件”（用0表示），我们就查看模型预测“非垃圾邮件”这一类别的概率。

交叉熵损失的公式是这样的：
$$
-\sum_{c=1}^{M} y_{o,c} \log(p_{o,c})
$$
其中，$M$ 是类别的数量，$y_{o,c}$ 是如果样本属于类别 $c$ 则为1，否则为0，$p_{o,c}$ 是我们模型预测样本属于类别 $c$ 的概率。

在二分类问题中，这个公式可以简化为：
$$
- y \log(p) + (1 - y) \log(1 - p)
$$
其中，$y$ 是真实标签，$p$ 是模型预测为正类的概率。

交叉熵损失函数的关键点在于，它惩罚了错误的预测。如果真实类别是1，模型预测的概率越低，损失就越大；相反，如果真实类别是0，模型预测的概率越高，损失也越大。所以，通过最小化交叉熵损失函数，我们可以训练我们的模型更准确地预测真实的类别。

In [ ]:
def cross_entropy(X, y, W):
    """
    X: N*(a+1), N个样本, 特征数量为为a, 外加1维偏置
    y: N*C, y为N个C维的one-hot向量
    W: (a+1)*C
    """
    p_y = softmax(X, W) # N*C, N个样本分别在C个类别的后验概率
    crossEnt = -torch.dot(y.reshape(-1), torch.log2(p_y).reshape(-1)) / y.shape[0]  # 展开成1维，点积
    return crossEnt

In [ ]:
X = torch.randn(10, 5)
y = torch.zeros(10, 3)
y[torch.arange(10), torch.randint(low=0, high=y.shape[1] - 1, size=(10,))] = 1  # 随机取1行中的某个元素为1，即确定1个样本对应的类别
W = torch.randn(5, 3)

In [ ]:
a = torch.arange(10)
b = torch.arange(10)

torch.sum(a * b), torch.dot(a,b)

In [ ]:
y  # 真实标签

In [ ]:
cross_entropy(X, y, W)

In [ ]:
prob_y = softmax(X, W)

In [ ]:
-torch.log2(prob_y)

In [ ]:
-torch.dot(y.reshape(-1), torch.log2(prob_y).reshape(-1)) / y.shape[0]

- 风险函数$\mathbf{R(W)}$关于$W$的梯度为
$$
\frac{\partial R(W)}{\partial W}=-\frac{1}{N}\sum_{n=1}^N\mathbf{x^{(n)}(y^{(n)}-\hat{y}^{(n)})}^T
$$

In [ ]:
def grad_crosEnt_W(X, y, W):
    '''
    X: N*(a+1), N个样本, 特征数量为为a, 外加1维偏置
    y: N*C, y为N个C维的one-hot向量
    W: (a+1)*C
    '''
    hat_y = softmax(X, W)
    a = (X.t() @ (y - hat_y)) / y.shape[0]  # (a+1)*N | N*C
    return -a

In [ ]:
grad_crosEnt_W(X, y, W)

## 4. 学习方法

- 输入: 训练集X，`one-hot`形式的标签y
- 输出：最优参数$w^*$
- 算法过程
    - 初始化$W_0:=0$，最大迭代次数$T$
    - 然后通过下式进行参数的迭代更新
    $$
    W_{t+1}:=W_t+\eta\left(\frac{1}{N}\sum_{n=1}^N\mathbf{x^{(n)}(y^{(n)}-\hat{y}^{(n)})}^T\right) 
    $$
    - 直到满足指定迭代次数，令$w^*=w^T$。

- 预测的正确率

In [ ]:
def precision_rate(X, y, W, X_with_bias=False):
    if X_with_bias:
        hat_X = X
    else:
        hat_X = torch.cat([X, torch.ones(X.shape[0], 1)], axis=1)  # 增广X
        
    pred_y = hat_y(hat_X, W)
    precision = torch.sum(pred_y[0] == torch.max(y, axis=1)[1]).numpy() / pred_y[0].numel()
    return precision

- 生成模拟数据

In [ ]:
X = torch.randn(1000, 8)
hat_X = torch.cat([X, torch.ones(X.shape[0], 1)], axis=1)  # 增广
true_W = torch.randn(hat_X.shape[1], 5)  # 增广
indices_y, y = hat_y(hat_X, true_W)

In [ ]:
train_X, train_y = X[:800], y[:800]
train_indices_y = indices_y[:800]
hat_train_X = hat_X[:800]
test_X, test_y = X[800:], y[800:]
test_indices_y = indices_y[800:]
hat_test_X = hat_X[800:]

### 4.1 方法1: 梯度下降-自己求导

In [ ]:
def softmax_sgd(X, y, num_steps=100, lr=0.1):
    '''
    X: N*(a+1), N个样本, 特征数量为为a, 外加1维偏置
    y: N*C, y为N个C维的one-hot向量
    W: (a+1)*C
    '''
    hat_X = torch.cat([X, torch.ones(X.shape[0], 1)], axis=1)  # 增广X
    W = torch.randn(hat_X.shape[1], y.shape[1])  # 增广参数矩阵
    for i in range(num_steps):
        W -= lr*grad_crosEnt_W(hat_X, y, W)
        loss = cross_entropy(hat_X, y, W)
        if (i+1) % 50 == 0:
            print(f'训练{i+1}轮, 交叉熵为{loss:.2f}')
            
    return W

In [ ]:
# 模拟数据
est_W = softmax_sgd(train_X, train_y, num_steps=1000)
precision_rate(train_X, train_y, est_W, X_with_bias=False), precision_rate(test_X, test_y, est_W, X_with_bias=False)

### 4.2 方法2: 随机梯度下降-自动求导

In [ ]:
def softmax_miniBatch_sgd(X, y, num_epoch=50, batch_size=40, lr=0.05):
    '''
    X: N*a, N个样本, 特征数量为为a
    y: N*C, y为N个C维的one-hot向量
    W: a*C
    '''
    hat_X = torch.cat([X, torch.ones(X.shape[0], 1)], axis=1)  # 增广X
    W = torch.randn(hat_X.shape[1], y.shape[1])  # 增广参数矩阵
    W.requires_grad_()
    dataset = TensorDataset(hat_X, y)
    data_iter = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)
    for epoch in range(num_epoch):
        for t_x, t_y in data_iter:
            l = cross_entropy(t_x, t_y, W)        
            l.backward()  # 计算损失函数在 W 上的梯度
            W.data.sub_(lr*W.grad/batch_size)
            W.grad.data.zero_()
            
        if (epoch + 1) % 50 == 0:
            with torch.no_grad():  # 不计算梯度，加速损失函数的运算
                train_l = cross_entropy(hat_X, y, W)  # 最近一次的负对数似然率
                est_W = W.detach().numpy()  # detach得到一个有着和原tensor相同数据的tensor
                print(f'epoch {epoch + 1}, loss: {train_l:.4f}')
            
    return est_W, train_l

In [ ]:
# 模拟数据
est_W, train_l = softmax_miniBatch_sgd(train_X, train_y, num_epoch=1000, batch_size=40, lr=0.1)
precision_rate(train_X, train_y, est_W), precision_rate(test_X, test_y, est_W)

### 4.3 方法3: 利用`torch.nn`构建模型，完成参数估计

- 定义类

In [ ]:
class SofmaxRegresModel(torch.nn.Module): 
    def __init__(self, dim_in, dim_out):
        # 首先找到LinearModel的父类torch.nn.Module，然后把类LinearModel的对象转换为类torch.nn.Module的对象, 
        # 即执行父类torch.nn.Module的初始化__init__()
        super(SofmaxRegresModel, self).__init__() 
        self.linear = torch.nn.Linear(dim_in, dim_out, bias=True)
        
    def forward(self, x):
        y_pred = self.linear(x)
        return torch.nn.functional.softmax(y_pred, dim=1)  # softmax

- 定义训练算法

In [ ]:
X.shape, y.shape

In [ ]:
dim_in = X.shape[1]
dim_out = y.shape[1]
# 实例化1个神经网络模型
net = SofmaxRegresModel(dim_in, dim_out)
# 初始化网络参数和偏置
net.linear.weight.data = torch.randn(dim_out, dim_in)
net.linear.bias.data = torch.zeros(dim_out)
# 损失函数
loss = torch.nn.CrossEntropyLoss()
# 随机梯度下降算法
trainer = torch.optim.SGD(net.parameters(), lr=0.05)

In [ ]:
# 加载数据
batch_size = 20
num_epochs = 200
dataset = TensorDataset(train_X, train_indices_y)
data_iter = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)
# 开始训练
for epoch in range(num_epochs):
    for t_x, t_y in data_iter:
        l = loss(net(t_x), t_y)  # 计算当前批量的交叉熵损失
        trainer.zero_grad()  # 参数梯度清零
        l.backward()  # 反向传播，计算梯度
        trainer.step()  # 更新参数
        
    if (epoch+1) % 20 == 0:
        with torch.no_grad():  # 不计算梯度，加速损失函数的运算
            l_epoch = loss(net(train_X), train_indices_y) 
            print('epoch {}, loss {}'.format(epoch+1, l_epoch)) 

- 结果

In [ ]:
w, b = net.parameters()
W = torch.cat([w.data, b.data.reshape(-1, 1)], axis=1)

pred_train_y = torch.max(net(train_X), axis=1)[1]
pred_test_y = torch.max(net(test_X), axis=1)[1]

print('train', torch.sum(pred_train_y == train_indices_y).numpy() / train_indices_y.numel())
print('test', torch.sum(pred_test_y == test_indices_y).numpy() / test_indices_y.numel())

## 5. 案例

鸢尾花数据集

In [ ]:
from sklearn import datasets
d = datasets.load_iris()

In [ ]:
print(d.DESCR)

In [ ]:
x_labels, y_labels = d['feature_names'], d['target_names']

In [ ]:
x_labels, y_labels

In [ ]:
rand_idx = np.arange(len(d['target']))  # 样本下标
np.random.shuffle(rand_idx)  # 打乱样本下标
t_idx = rand_idx[:100]  # 训练集下标
v_idx = rand_idx[100:]  # 测试集下标
x_train, y_train = torch.from_numpy(d['data'][t_idx]).type(torch.FloatTensor), torch.from_numpy(d['target'][t_idx])  # 训练集
x_valid, y_valid = torch.from_numpy(d['data'][v_idx]).type(torch.FloatTensor), torch.from_numpy(d['target'][v_idx])  # 测试集

onehot_y_train = torch.zeros(x_train.shape[0], 3)  # 训练集标签转为one-hot向量
onehot_y_train[torch.arange(x_train.shape[0]), y_train] = 1
onehot_y_valid = torch.zeros(x_valid.shape[0], 3)  # 预测集标签转为one-hot向量
onehot_y_valid[torch.arange(x_valid.shape[0]), y_valid] = 1

In [ ]:
y_train

In [ ]:
onehot_y_train

### 5.1 方法1

In [ ]:
est_W = softmax_sgd(x_train, onehot_y_train, num_steps=1000)
print(f"Train accuracy rate: {precision_rate(x_train, onehot_y_train, est_W)}")
print(f"Valid accuracy rate: {precision_rate(x_valid, onehot_y_valid, est_W)}")

### 5.2 方法2 

In [ ]:
# 鸢尾花
est_W, _ = softmax_miniBatch_sgd(x_train, onehot_y_train, num_epoch=1000, batch_size=40, lr=0.1)
print(f"Train accuracy rate: {precision_rate(x_train, onehot_y_train, est_W)}")
print(f"Valid accuracy rate: {precision_rate(x_valid, onehot_y_valid, est_W)}")

### 5.3 方法3

In [ ]:
# 实例化
dim_in = 4  # 特征数量
dim_out = 3  # 类别数量
net = SofmaxRegresModel(dim_in, dim_out)
# 初始化网络参数和偏置
net.linear.weight.data = torch.randn(dim_out, dim_in)
net.linear.bias.data = torch.randn(dim_out)
# 损失函数
loss = torch.nn.CrossEntropyLoss()
# 随机梯度下降算法
trainer = torch.optim.SGD(net.parameters(), lr=0.03)
# 加载数据
batch_size = 100
num_epochs = 2000
dataset = TensorDataset(x_train, y_train)
data_iter = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# 训练
for epoch in range(num_epochs):
    for t_x, t_y in data_iter:
        l = loss(net(t_x), t_y)  # 计算当前批量的交叉熵损失
        trainer.zero_grad()  # 参数梯度清零
        l.backward()  # 反向传播，计算梯度
        trainer.step()  # 更新参数
    if (epoch+1) % 50 == 0:
        with torch.no_grad():  # 不计算梯度，加速损失函数的运算
            l_epoch = loss(net(x_train), y_train) 
            print('epoch {}, loss {}'.format(epoch+1, l_epoch)) 

In [ ]:
w, b = net.parameters()
W = torch.cat([w.data, b.data.reshape(-1, 1)], axis=1)

In [ ]:
pred_y_train = torch.max(net(x_train), axis=1)[1]
pred_y_valid = torch.max(net(x_valid), axis=1)[1]

In [ ]:
pred_y_train

In [ ]:
torch.sum(pred_y_train == y_train).numpy() / y_train.numel()

In [ ]:
torch.sum(pred_y_valid == y_valid).numpy() / y_valid.numel()

## 附. 熵相关概念

一条信息的信息量大小和它的不确定性有很大的关系。一句话如果能带来很多让我们不知道的内容，我们就称这句话的信息量比较大。比如你听到“云南西双版纳下雪了”，如果你要去验证这句话的真假，则需要去看天气预报、问当地人等等查证（因为云南西双版纳从没下过雪）。相反，如果和你说“人一天要吃三顿饭”，那这条信息的信息量就很小，因为这条信息的确定性很高，我们不需要用很多信息取证明它。因此，可将事件$x_0$的信息量表示为：
$$
I(x_0)=-\log p(x_0)
$$
> 一条信息信息量的大小与其发生概率（不确定性越小，发生概率越大）呈反比，发生概率越大，信息量越小。

### A1. 熵
信息量是对于单个事件来说的，但是实际情况一件事有很多种发生的可能，比如掷骰子有可能出现6种情况，明天的天气可能晴、多云或者下雨等等。因此，我们需要评估事件对应的所有可能性。

熵（entropy）是表示随机变量不确定的度量，是对表征所有可能发生的事件所需信息量的期望。

设$X$是一个取有限个值的随机变量，其概率分布为
$$
P(X=x_i)=p_i,i=1,2,...,n
$$
熵定义为
$$
H(x)=\sum_{i=0} p(x_i) I(x_i)=-\sum_{i=1}^n p(x_i) \log p(x_i)
$$
上式中，若$p_i=0$，则定义$0\log 0=0$；对数以2或者e为底，这时熵的单位分别称为比特(bit)或者纳特(nat)。熵只依赖于$X$的分布，与其取值无关，因此也可将$X$的熵记作$H(p)$, 即
$$
H(p)=-\sum_{i=1}^n p_i \log p_i
$$
熵越大，不确定越大。

In [ ]:
def entropy(P):
    '''
    P为概率分布
    '''
    return -np.sum([p*np.log2(p) if p > 0 else 0 for p in P])

In [ ]:
P1 = np.ones(10) / 10
P2 = np.zeros(10)
P2[3] = 1
entropy(P1), entropy(P2)

In [ ]:
P1

In [ ]:
P3 = np.array([0, 0.1, 0.1, 0.1, 0.2, 0.2, 0.1, 0.1, 0.1, 0])

In [ ]:
entropy(P3)

In [ ]:
P2

### A2. 条件熵
条件熵(conditional entropy): 表示在已知随机变量$X$的条件下随机变量$Y$的不确定性。

$$
H(Y|X)=\sum_{i=1}^n P(X=x_i)H(Y|X=x_i)
$$

其中，$H(Y|X=x_i)=-\sum_j P(Y=y_j|X=x_i)\log P(Y=y_j|X=x_i)$，表示在$X=x_i$时Y的不确定程度；$p(Y=y_j|X=x_i) = \frac{p(X=x_i, Y=y_j)}{p(X=x_i)}$。
> 如果X与Y无关，则有$H(Y|X)=H(Y)$；如果Y由X唯一决定，则有$H(Y|X)=0$

In [ ]:
def conditional_entropy(P_XY):
    '''
    P_XY为X和Y的联合概率分布shape(x_size, y_z)
    '''
    return np.sum([np.sum(P_XY[i]) * entropy(P_XY[i, :]/np.sum(P_XY[i])) 
                   for i in P_XY.shape[1]])

### A3. KL散度（相对熵）

相对熵(`relative entropy`)或KL散度(`Kullback-Leibler divergence`)：度量一个概率分布$p(x)$相对另一个概率分布$q(X)$的差异

$$
\text{KL(p||q)}=-\sum_x p(x)\log\frac{q(x)}{p(x)}
$$

由`Jesen`不等式可证明，$\text{KL(p||q)}\geq 0$，当且仅当对于所有$x$有$p(x)=q(x)$时，取等号。

此外，需注意，$\text{KL(p||q)}\neq \text{KL(q||p)}$

> 注意，要讨论p_x[i] == 0时候p_x[i]*np.log(q_x[i]/p_x[i]) 的计算问题

In [ ]:
def KL(p_x, q_x):
    return -np.sum([p_x[i]*np.log(q_x[i]/p_x[i]) if p_x[i] > 0 and q_x[i] > 0 else 0 
                    for i in range(len(p_x))])

In [ ]:
KL(P1, P1), KL(P2, P1)

### A4. 交叉熵

交叉熵定义如下:

$$
\text{crossEntropy(p(x), q(x))} = -\sum_x p(x)\log q(x)
$$

- 与KL散度的关系
$$
\begin{aligned}
\text{KL(p||q)} &= -\sum_x p(x)\log\frac{q(x)}{p(x)}\\
&= -\sum_x p(x)\log q(x) + \sum_x p(x)\log p(x) \\
&= \text{crossEntropy(p(x), q(x))} - H\left(p(x)\right)
\end{aligned}
$$

即有$\text{crossEntropy(p(x), q(x))} = \text{KL(p||q)} + H\left(p(x)\right)$

由于$H\left(p(x)\right)$为定值，针对q最小化交叉熵等价于最小化`KL(p||q)`，即使理论分布与抽样分布之间的差异最小。

In [ ]:
def cross_entropy(p_x, q_x):
    return -np.sum([p_x[i]*np.log(q_x[i]) if q_x[i] > 0 else 0 for i in range(len(p_x))])

In [ ]:
cross_entropy(P1, P2), cross_entropy(P1, P1), cross_entropy(P2, P2)

## 参考资料
1. 李航. 统计学习方法. 2017.
2. 邱锡鹏. 神经网络与机器学习. 2020.